In [ ]:
from htbac import Runner, Protocol, Simulation, System, AbFile, DataAggregate
from htbac.protocols import RFE, ESMACS # Allows users to utilize pre-defined protocols 

In [ ]:
export RADICAL_PILOT_DBURL=<mongodb://>
export SAGA_PTY_SSH_TIMEOUT=2000
export RADICAL_PILOT_PROFILE=True
export PATH=/usr/sbin:$PATH
export RADICAL_VERBOSE="DEBUG"
export RADICAL_ENTK_PROFILE=True

In [ ]:
# define system
pdb = AbFile('systems/ptp1b-l1-l2-complex.pdb', tag='pdb')
top = AbFile('systems/ptp1b-l1-l2-complex.top', tag='topology')
tag = AbFile('systems/ptp1b-l1-l2-tags.pdb', tag='alchemicaltags')
cor = AbFile('systems/ptp1b-l1-l2-complex.inpcrd', tag='coordinate')
system = System(name='ptp1b-l1-l2', files=[pdb, top, tag, cor])

In [ ]:
# Create customized simulation steps for relative free energy protocol

# define protocol:
p = Protocol()

# define step 0: 
s0 = Simulation(name='minimizer')
s0.engine = 'namd'
s0.add_ensemble('replica', range(5))
s0.add_ensemble('lambdawindow', [1.0, 0.5, 0.0]) 
s0.add_input_file("default_configs/rfe/ties-0.conf", is_executable_argument=True)
s0.system = system

# set all values that have <placeholder> in the *.conf file 

s0.cutoff = 10.0
s0.switchdist = 8.0
s0.pairlistdist = 11.5
s0.numsteps = 5000
s0.watermodel = "tip3"

# append step 0 to protocol:
p.append(s0)

# define step 1: 
s1 = Simulation(name='equilibrate')
s1.engine = 'namd'
s1.add_ensemble('replica', range(5))
s1.add_ensemble('lambdawindow', [1.0, 0.5, 0.0])
s1.add_input_file("default_configs/rfe/ties-1.conf", is_executable_argument=True)
s1.system = system

# <placeholders>
s1.cutoff = 10.0
s1.switchdist = 8.0
s1.pairlistdist = 11.5
s1.numsteps = 50000 

# append step 1 to protocol:
p.append(s1)

ht = Runner('titan_orte', comm_server=('two.radical-project.org', 33048))
ht.add_protocol(p)
ht.run(walltime=480)


In [ ]:
# using a pre-defined protocol (RFE), add details to the simulation steps

# Protocol `p` is made up of 2 simulation steps, 1 aggregate data:
# minimize -> MD -> DataAggregate

p = Protocol(RFE.minimize(),
             RFE.simulation(),
             DataAggregate(extension=".alch"))

step_counts = [1000, 1000]

for rfe, numsteps in zip(p.simulations(), step_counts):
    rfe.system = system
    rfe.engine = 'namd'
    rfe.processes = 32
    rfe.threads_per_process = 1

    rfe.cutoff = 10.0
    rfe.switchdist = 8.0
    rfe.pairlistdist = 11.5
    rfe.numsteps = numsteps
    rfe.watermodel = 'tip3'

    rfe.add_ensemble('replica', range(5))
    rfe.add_ensemble('lambdawindow', [0.0, 0.5, 1.0])

    
ht = Runner('bw_aprun', comm_server=('two.radical-project.org', 33048))
ht.add_protocol(p)
ht.run(walltime=480, queue='high')
